<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Rodrigo Guerra
- Nombre de alumno 2: Mariano Suárez


### **Link de repositorio de GitHub:** `https://github.com/MarianoSuarez03/Laboratorios-Python`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

#Importamos librerias utiles 😸

In [28]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression



# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

Requirement already up-to-date: plotly in /root/venv/lib/python3.7/site-packages (5.4.0)
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [36]:
df_players = pd.read_csv('/work/Laboratorio7_enunciado/stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [39]:
df_players.loc[(df_players['National_Position'].isna()),'target'] = 0
df_players.loc[~(df_players['National_Position'].isna()),'target'] = 1

l = df_players.count()[0]

df_class = df_players.groupby('target').size().reset_index().rename({0:'Count'},axis=1)
df_class['Porcentaje'] = (df_class['Count']/l)*100

display(df_class)

,target,Count,Porcentaje
0,0.0,16513,93.887878
1,1.0,1075,6.112122


Se observa que gran parte de los jugadores corresponde a no seleccionados, representando un 93.9% del total de la muestra, solamente un 6.1% son convocados a la selección de su país.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [41]:
### Código aquí ###

preprocessing_transformer = ColumnTransformer(
    transformers=[
        (
            "OneHotEncoder",
            OneHotEncoder(drop='first', handle_unknown='ignore'),
            [
                "Club_Position",
                'Nationality',
                "Preffered_Foot",
                "Skill_Moves",
                "Work_Rate",
                "Weak_foot"
            ],
        ),
        (
            "StandardScaler",
            StandardScaler(),
            [
                "Weight",
                "Height",
                "Aggression",
                "Reactions",
                "Vision",
                "Composure",
                "Acceleration",
                "Speed",
                "Balance",
                "Strength",
                "Agility",
                "Jumping",
                "Finishing",
                "Volleys",
                "Penalties"
            ],
        ),
        (
            "MinMaxScaler",
            MinMaxScaler(),
            [
                "Ball_Control",
                "Dribbling",
                "Marking",
                "Sliding_Tackle",
                "Standing_Tackle",
                "Interceptions",
                "Crossing",
                "Short_Pass",
                "Long_Pass",
                "Stamina",
                "Heading",
                "Shot_Power",
                "Curve",
                "Freekick_Accuracy",
                "Long_Shots"
            ],
        ),
    ]
)


linear_svc_pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer),
        ("linear_SVM", SVC(kernel="linear")),
    ]
)

svc_pipe = Pipeline(
    [("preprocesamiento", preprocessing_transformer), ("SVM", SVC(kernel="rbf"))]
)

knn_pipe = Pipeline(
    [("preprocesamiento", preprocessing_transformer), ("KNN", KNeighborsClassifier())]
)


 Para la construcción del Pipeline se procede a utilizar *ColumnTransformer* para realizar el preprocesamiento, así como los modelos de Support Vector Machine (con kernel lineal y rbf) y Vecinos Más Cercanos (KNN).

 El preprocesamiento utiliza 3 transformaciones distintas, dependiendo de la características de los atributos, para las variables categóricas se realiza la transformación con *OneHotEncoder*, mientras que para las variables numéricas se utiliza *StandardScaler* o *MinMaxScalar* dependiendo si el atributo tiene posee o no una distribución normal, respectivamente.

 Los clasificadores son escogidos a través de la guía, siguiendo el flujo para predecir una categoría (clasificación binaria) con data etiquetada y un número de muestras menor a 100 mil.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [43]:
### Código aquí ###

df_y = df_players["target"]
df_X = df_players.drop(["Name", "National_Position", "target"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.33, random_state=42, stratify=df_y
)

linear_svc = linear_svc_pipe.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
print('Resultados Linear SVC:')
print(classification_report(y_pred, y_test))

rbf_svc = svc_pipe.fit(X_train, y_train)
y_pred = rbf_svc.predict(X_test)
print('Resultados RBF SVC')
print(classification_report(y_pred, y_test))


knn = knn_pipe.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Resultados KNN')
print(classification_report(y_pred, y_test))



/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros

Resultados Linear SVC:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      5681
           1       0.24      0.69      0.36       124

    accuracy                           0.95      5805
   macro avg       0.62      0.82      0.67      5805
weighted avg       0.98      0.95      0.96      5805

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros

Resultados RBF SVC
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      5804
           1       0.00      1.00      0.01         1

    accuracy             

**Respuesta**:

```
Se observa que los resultado para los clasificadores son malos, si bien la precision en la clase 0 es muy buena, se debe a que las clases están muy desbalanceadas, entonces los modelos predicen casi en un 100% o en un 100% clase 0, y entonces los resultados en esa clase son muy buenos, y en la clase 1 son muy malos. 
Es posible mejorar los resultados realizando un entrenamiento con pesos, donde los pesos estén en una proporcion similar a la proporcion de clases, o también se pueden hacer subsamples y oversamples de las clases 0 y 1 respectivamente, para asi tener un mejor balance de clases.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [48]:
### Código aquí ###

cp = {
    "ST": "ataque",
    "CF": "ataque",
    "RW": "central_ataque",
    "CAM": "central_ataque",
    "LW": "central_ataque",
    "RM": "central",
    "CM": "central",
    "LM": "central",
    "RWB": "central_defensa",
    "CDM": "central_defensa",
    "LWB": "central_defensa",
    "RB": "defensa",
    "CB": "defensa",
    "LB": "defensa",
    "GK": "arquero",
}

df_positions = df_players.copy()
df_positions["Position"] = df_positions["Club_Position"].map(cp).astype('category')
df_plot = (
    df_positions.groupby("Position").size().reset_index().rename({0: "Count"}, axis=1)
)

display(df_plot)

# fig = px.bar(df_plot, x="Position", y="Count")
# fig.show()


df_positions = df_positions.dropna(subset = ['Position']).drop(['National_Position', 'Club_Position', 'target'], axis = 1)
df_positions

,Position,Count
0,arquero,632
1,ataque,430
2,central,907
3,central_ataque,581
4,central_defensa,209
5,defensa,1180


,Name,Nationality,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Position
0,Cristiano Ronaldo,Portugal,185,80,Right,32,High / Low,4,5,93,...,95,85,92,93,90,81,76,85,88,central_ataque
1,Lionel Messi,Argentina,170,72,Left,29,Medium / Medium,4,4,95,...,68,71,85,95,88,89,90,74,85,central_ataque
2,Neymar,Brazil,174,68,Right,25,High / Medium,5,5,95,...,61,62,78,89,77,79,84,81,83,central_ataque
3,Luis Suárez,Uruguay,182,85,Right,30,High / Medium,4,4,91,...,69,77,87,94,86,86,84,85,88,ataque
4,Manuel Neuer,Germany,193,92,Right,31,Medium / Medium,4,1,48,...,78,25,25,13,16,14,11,47,11,arquero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17298,Raymond Skjelde,Norway,185,75,Right,18,Medium / Medium,2,2,29,...,65,44,35,22,25,24,22,41,31,defensa
17301,Andrew O'Connor,Republic of Ireland,175,73,Left,21,Medium / Medium,3,2,37,...,55,43,26,28,29,30,30,31,20,defensa
17310,Tom Bolarinwa,England,180,80,Right,26,Medium / Medium,3,3,42,...,54,22,38,37,37,37,32,29,29,central
17312,Zak Mills,England,182,83,Right,24,Medium / Medium,4,2,49,...,61,43,22,24,28,30,32,39,31,defensa


In [45]:
ct = ColumnTransformer(
    transformers=[
        (
            "OneHotEncoder",
            OneHotEncoder(drop='first', handle_unknown='ignore'),
            [
                'Nationality',
                "Preffered_Foot",
                "Skill_Moves",
                "Work_Rate",
                "Weak_foot"
            ],
        ),
        (
            "StandardScaler",
            StandardScaler(),
            [
                "Weight",
                "Height",
                "Aggression",
                "Reactions",
                "Vision",
                "Composure",
                "Acceleration",
                "Speed",
                "Balance",
                "Strength",
                "Agility",
                "Jumping",
                "Finishing",
                "Volleys",
                "Penalties"
            ],
        ),
        (
            "MinMaxScaler",
            MinMaxScaler(),
            [
                "Ball_Control",
                "Dribbling",
                "Marking",
                "Sliding_Tackle",
                "Standing_Tackle",
                "Interceptions",
                "Crossing",
                "Short_Pass",
                "Long_Pass",
                "Stamina",
                "Heading",
                "Shot_Power",
                "Curve",
                "Freekick_Accuracy",
                "Long_Shots"
            ],
        ),
    ]
)


linear_svc_pipe = Pipeline(
    [
        ("preprocesamiento", ct),
        ("linear_SVM", SVC(kernel="linear")),
    ]
)

svc_pipe = Pipeline(
    [("preprocesamiento", ct), ("SVM", SVC(kernel="rbf"))]
)

knn_pipe = Pipeline(
    [("preprocesamiento", ct), ("KNN", KNeighborsClassifier())]
)



In [46]:
df_y = df_positions.loc[:,"Position"]
df_X = df_positions.drop(["Name", "Position"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.33, random_state=42, stratify=df_y
)

linear_svc = linear_svc_pipe.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
print('Resultados Linear SVC:')
print(classification_report(y_pred, y_test))

rbf_svc = svc_pipe.fit(X_train, y_train)
y_pred = rbf_svc.predict(X_test)
print('Resultados RBF SVC')
print(classification_report(y_pred, y_test))


knn = knn_pipe.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Resultados KNN')
print(classification_report(y_pred, y_test))


/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros

Resultados Linear SVC:
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.75      0.74      0.75       144
        central       0.61      0.55      0.57       332
 central_ataque       0.31      0.43      0.36       138
central_defensa       0.13      0.31      0.18        29
        defensa       0.95      0.83      0.88       448

       accuracy                           0.72      1300
      macro avg       0.63      0.64      0.63      1300
   weighted avg       0.76      0.72      0.74      1300

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [0] during transform. These unknown cat

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
Se observa que los mismos modelos para un problema distinto al anterior, muestran mejores resultados, donde en las clases con mayor frecuencia, se alcanzan mejores métricas, mientras que en las clases más chicas que son 'central_ataque' y 'central_defensa' presentan las peores métricas en cada clasificador. 
El clasificador con mejores resultados si es que se utiliza como parámetro macro avg, es decir, las clases son igual de importantes, entonces es el promedio de la métrica en cada una de las clases, sería un SVC, dependiendo del objetivo del modelo se elegería la mejor métrica según el caso.
Los resultados se pueden mejorar utilizando Grid Search, para buscar los hiperparámetro que mejor se ajustan al problema. Además, se puede intentar balancear las clases y realizar un entrenamiento acorde a los pesos de cada clase.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [8]:
### Código aquí ###
df_salary = pd.read_csv('/work/Laboratorio7_enunciado/salarios.csv')
df_salary = df_salary.loc[:,['Player', 'Weekly Salary']].rename({'Player': 'Name'}, axis = 1)
df_wage = df_salary.merge(df_players, on='Name')
df_wage['Weekly Salary'] = df_wage['Weekly Salary'].astype(int)


In [9]:
df_wage

,Name,Weekly Salary,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,target
0,Lionel Messi,1538905,Argentina,RW,RW,170,72,Left,29,Medium / Medium,...,68,71,85,95,88,89,90,74,85,1
1,Cristiano Ronaldo,1248536,Portugal,LS,LW,185,80,Right,32,High / Low,...,95,85,92,93,90,81,76,85,88,1
2,Antoine Griezmann,997032,France,CAM,RS,176,67,Left,26,High / Medium,...,87,80,81,88,82,84,75,71,87,1
3,Neymar,797726,Brazil,LW,LW,174,68,Right,25,High / Medium,...,61,62,78,89,77,79,84,81,83,1
4,Gareth Bale,607706,Wales,RS,RW,183,74,Left,27,High / Medium,...,85,86,91,87,90,86,85,76,76,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Michael Langer,903,Austria,NaN,GK,194,92,Right,32,Medium / Medium,...,50,14,15,14,13,12,12,13,11,0
1862,Nicolas Lemaître,859,France,NaN,Res,185,83,Right,20,Medium / Medium,...,32,23,23,17,15,19,14,22,17,0
1863,Andrea Pinamonti,805,Italy,NaN,Res,181,72,Right,17,Medium / Low,...,65,62,57,74,60,43,43,69,43,0
1864,Tim Wiesner,674,Germany,NaN,Res,192,87,Right,20,Medium / Medium,...,57,16,19,13,15,18,16,20,14,0


In [24]:
from sklearn.ensemble import RandomForestRegressor

ct = ColumnTransformer(
    transformers=[
        (
            "OneHotEncoder",
            OneHotEncoder(drop='first', handle_unknown='ignore'),
            [
                'Nationality',
                "Preffered_Foot",
                "Skill_Moves",
                "Work_Rate",
                "Weak_foot",
                # "target",
                # 'Club_Position'
            ],
        ),
        (
            "StandardScaler",
            StandardScaler(),
            [
                # "Weight",
                # "Height",
                'Age',
                "Aggression",
                "Reactions",
                "Vision",
                "Composure",
                "Acceleration",
                "Speed",
                "Balance",
                "Strength",
                "Agility",
                "Jumping",
                "Finishing",
                "Volleys",
                "Penalties"
            ],
        ),
        (
            "MinMaxScaler",
            MinMaxScaler(),
            [
                "Ball_Control",
                "Dribbling",
                # "Marking",
                # "Sliding_Tackle",
                # "Standing_Tackle",
                # "Interceptions",
                "Crossing",
                "Short_Pass",
                "Long_Pass",
                "Stamina",
                "Heading",
                "Shot_Power",
                "Curve",
                "Freekick_Accuracy",
                "Long_Shots"
            ],
        ),
    ]
)


lm_pipe = Pipeline(
    [
        ("preprocesamiento", ct),
        ("RandomForest", RandomForestRegressor()),
    ]
)



In [47]:
df_y = df_wage.loc[:, "Weekly Salary"]
df_X = df_wage.drop(
    [
        "Name",
        "National_Position",
        "target",
        # "Skill_Moves",
        # "Work_Rate",
        # "Weight",
        # "Height",
        "Marking",
        "Sliding_Tackle",
        "Standing_Tackle",
        "Interceptions",
        "Club_Position"
    ],
    axis=1,
)

X_train, X_test, y_train, y_test = train_test_split(
    df_wage, df_y, test_size=0.33, random_state=42
)

lm = lm_pipe.fit(X_train, y_train)
# y_pred = lm.predict(X_test)

print("Resultados R^2:")
print(lm.score(X_test, y_test))
# y_pred = lm.predict(X_test)

# from sklearn.metrics import r2_score

# r2_score(y_test, y_pred)


Resultados R^2:
0.38042754673551593
/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros



**Respuesta**:

* La métrica $R^2$ representa la proporción en la que varia la variable dependiente, en este caso el salario semanal de los jugadores, explicada a través de las variables independientes del modelo.


* El prepocesamiento se realiza de manera similar a los casos anteriores, y se elige como regresor el *RandomForestRegressor* el cual se encuentra al final del flujo de la guía de elección de estimadores, al predecir una cantidad (salarios semanales) con una muestra menor a 100 mil observaciones, utilizando la mayoría de los atributos. El regresor de árboles de desición ajusta una serie de árboles de desición en distintas submuestras, utilizando los promedios para mejorar la capacidad predictiva y controlar el sobreajuste del modelo.


* Como se puede apreciar, se obtiene un $R^2$ de 0.38, es decir de la variación presente en los sueldos semanales de los jugadores, el modelo explica un 38% a través de las variables utilizadas. Si bien el resultado es aceptable, no se puede predecir con un gran nivel de confianza el sueldo semanal de los jugadores.


* Es posible mejorar los resultados del modelo, incluyendo variables explicativas faltantes dentro este, como por ejemplo el equipo en el que juega, o la liga a la que pertenece, debido a que ciertas ligas y equipos poseen mayor poder adquisitivo que otras. Mas observaciones ayudarían tambien, para mejorar el desempeño del modelo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1d355b97-bbb9-4f56-823f-44a85d475e0a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>